## imports

In [1]:
import sys 
from collections import defaultdict
# genequest imports 
from genequest.io_handler.parser import FastaReader
from genequest.common.errors import *
from genequest.analysis.assembler import run_de_brujin, kmerize


## Script Prototype

### inputs 

In [2]:
reads = "../data/READS.fasta"
query = "../data/QUERY.fasta"
kmer_size = 20
match_score = 10
gap_score = 0
mismatch_score = -5

### Main script 

In [3]:
# loading in the read and gene_query
entries = FastaReader(reads)
gene_query = FastaReader(query)

print("Loaded Sequence information:")
print("Query sequence length: {}".format(len(gene_query[0])))
print("Total number of reads: {}".format(entries.n_entries))
print("Scaffolds: {}".format(entries.scaffold_ids))
print("Total number of scaffolds: {}".format(entries.n_scaffolds))

# grouped_entries = entries.group_by_scaffold()


Loaded Sequence information:
Query sequence length: 648
Total number of reads: 124520
Scaffolds: ['2G5Z3', '2S43D']
Total number of scaffolds: 2


In [10]:
data = run_de_brujin(entries, k=20)

5098878

In [6]:
s = "ATCG"

In [7]:
s[:len(s)-1]

'ATC'